# Our SAWYER Process

# Step 3: Use Reinforcement Learning to align SAWYER to preferred responses

In [ ]:
import os
import pandas as pd
import torch
from datasets import load_dataset
from tqdm import tqdm
from transformers import AutoTokenizer, HfArgumentParser, pipeline, AutoModelForSequenceClassification

from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed

from datasets import Dataset
tqdm.pandas()

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

device = torch.device("cuda")

device

device(type='cuda')

In [ ]:

config = PPOConfig(
    model_name='sawyer_rl',
    learning_rate=1.4e-6,
    batch_size=4,
    gradient_accumulation_steps=1,
    ppo_epochs=1,
    seed=42,
    log_with="wandb",
    optimize_cuda_cache=True
)


In [ ]:
dolly = load_dataset('databricks/databricks-dolly-15k')
dolly

Using custom data configuration databricks--databricks-dolly-15k-6e0f9ea7eaa0ee08
Found cached dataset json (C:/Users/sinan/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-6e0f9ea7eaa0ee08/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 15011
    })
})

In [ ]:
dolly['train'][0]

{'instruction': 'When did Virgin Australia start operating?',
 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.",
 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.',
 'category': 'closed_qa'}

In [ ]:
# load our custom tokenizer
tokenizer = AutoTokenizer.from_pretrained("./sawyer/tokenizer/")
tokenizer.pad_token, tokenizer.pad_token_id

('<PAD>', 50259)

In [ ]:
QUERY_KEY = "Question:"
RESPONSE_KEY = "Response:"
PAD_KEY = "<PAD>"


In [ ]:
dolly

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 15011
    })
})

In [ ]:
# quick cleaning function to clean some text
def ascci(text): return ''.join(char if ord(char) < 128 else '?' for char in text)


def preprocess_function(examples):
    new_examples = {
        "input_ids": [],
        "attention_mask": [],
        "input_text": []
    }
    for instruction, context in zip(examples["instruction"], examples['context']):
        if context:
            # make sure this is the same structure as reward model
            input_text = QUERY_KEY+ ' ' + instruction + '\n\n' + context + '\n' + RESPONSE_KEY
        else:
            input_text = QUERY_KEY+ ' ' + instruction + '\n'+RESPONSE_KEY
        tokenized_question = tokenizer(input_text, return_tensors='pt')
        new_examples['input_text'].append(input_text)
        new_examples["input_ids"].append(tokenized_question["input_ids"])
        new_examples["attention_mask"].append(tokenized_question["attention_mask"])
    return new_examples

dolly = dolly.map(preprocess_function, batched=True, batch_size=64)

Loading cached processed dataset at C:/Users/sinan/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-6e0f9ea7eaa0ee08/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab\cache-cd1f7edf0bb666e3.arrow


In [ ]:
dolly.set_format(type="torch")

In [ ]:
dolly = dolly.filter(lambda x: x["input_ids"].shape[-1] < 512, batched=False)


Loading cached processed dataset at C:/Users/sinan/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-6e0f9ea7eaa0ee08/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab\cache-81e9227c545cf6c4.arrow


In [ ]:
dolly

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category', 'input_ids', 'attention_mask', 'input_text'],
        num_rows: 14559
    })
})

In [ ]:
dolly['train'][0]

{'instruction': 'When did Virgin Australia start operating?',
 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.",
 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.',
 'category': 'closed_qa',
 'input_ids': tensor([[50257,  1649,   750,  5283,  4505,   923,  5361,    30,   198,   198,
          34674,  4505,    11,   262,  7313,  1438,   286,  5283,  4505, 16712,
            350,   774, 12052,    11,   318,   281,  6638,    12,  3106, 18091,
          

In [ ]:
reward_model_name = 'sawyer_rm' # From our rewards notebook

reward_model = AutoModelForSequenceClassification.from_pretrained(
    reward_model_name, num_labels=1,
)
reward_tokenizer = AutoTokenizer.from_pretrained('roberta-base')  # our tokenizer isn't different from base roberta

In [ ]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

# set seed before initializing value head for deterministic eval
set_seed(42)

model = AutoModelForCausalLMWithValueHead.from_pretrained(
    'sawyer',
)

# We then build the PPOTrainer, passing the model, the reference model, the tokenizer
ppo_trainer = PPOTrainer(
    config,
    model,
    ref_model=create_reference_model(model),
    tokenizer=tokenizer,
    dataset=dolly['train'],
    data_collator=collator,
)

wandb: Currently logged in as: profoz. Use `wandb login --relogin` to force relogin


In [ ]:
# We then define the arguments to pass to the `generate` function. These arguments
# are passed to the `generate` function of the PPOTrainer, which is a wrapper around
# the `generate` function of the trained model.
generation_kwargs = {
    "do_sample": True,
    "pad_token_id": tokenizer.pad_token_id,
    "eos_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 128,
}

output_length_sampler = LengthSampler(32, 1024)

In [ ]:
dolly['train'][0]['input_ids'], tokenizer.batch_decode(dolly['train'][0]['input_ids'])

(tensor([[50257,  1649,   750,  5283,  4505,   923,  5361,    30,   198,   198,
          34674,  4505,    11,   262,  7313,  1438,   286,  5283,  4505, 16712,
            350,   774, 12052,    11,   318,   281,  6638,    12,  3106, 18091,
             13,   632,   318,   262,  4387, 18091,   416, 11026,  2546,   284,
            779,   262,  5283,  4508,    13,   632, 32400,  2594,   319,  3261,
           2932,  4751,   355,  5283,  4518,    11,   351,   734,  6215,   319,
            257,  2060,  6339,    13,   632,  6451,  1043,  2346,   355,   257,
           1688, 18091,   287,  4505,   338,  5928,  1910,   706,   262,  9807,
            286,  1052, 17744,  4505,   287,  2693,  5878,    13,   383, 18091,
            468,  1201,  7334,   284,  3264,  4691,  3933,  4736,   287,  4505,
             11,   422, 38459,   287, 26914,    11, 14819,   290, 11852,    13,
            198, 50258]]),
 ["Question: When did Virgin Australia start operating?\n\nVirgin Australia, the trading name

In [ ]:
dolly['train'][0]

{'instruction': 'When did Virgin Australia start operating?',
 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.",
 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.',
 'category': 'closed_qa',
 'input_ids': tensor([[50257,  1649,   750,  5283,  4505,   923,  5361,    30,   198,   198,
          34674,  4505,    11,   262,  7313,  1438,   286,  5283,  4505, 16712,
            350,   774, 12052,    11,   318,   281,  6638,    12,  3106, 18091,
          

In [ ]:
response_token_id = tokenizer.vocab[RESPONSE_KEY]

response_token_id

50258

In [ ]:
EPOCHS=2

In [ ]:
from tqdm.auto import tqdm
steps = 0
for epoch in tqdm(range(EPOCHS)):
    for batch in tqdm(ppo_trainer.dataloader):
        batch['response'] = []
        batch['query'] = []
        batch['rewards'] = []
        response_tensors = []
        for input_ids in batch["input_ids"]:
            batch['query'].append(tokenizer.batch_decode(input_ids, skip_special_tokens=True)[0])
            response_tensor = ppo_trainer.generate(
                input_ids.squeeze(), **generation_kwargs,
            )
            indices = torch.where(response_tensor == response_token_id)
            try:
                first_occurrence_index = indices[1][0].item()
            except:
                print('failure to find first_occurrence_index', response_tensor)
                print(tokenizer.batch_decode(response_tensor))
                first_occurrence_index = 0
            # Truncate the tensor grabbing all elements after the first occurrence
            response_tensor = response_tensor[:, first_occurrence_index+1:]
            if response_tensor.ndim == 0:
                response_tensor = response_tensor.unsqueeze(0).unsqueeze(0)
            if response_tensor.shape[-1] < 2:
                response_tensor = response_tensor.repeat(1, 4)
            batch['response'].append(tokenizer.batch_decode(response_tensor, skip_special_tokens=True)[0])
            response_tensors.append(response_tensor.squeeze())
            t = reward_tokenizer(batch['query'][-1].replace('Response:', '').strip(), batch['response'][-1], return_tensors='pt', truncation=True)
            try:
                r = reward_model(**t)
            except:
                print(t['input_ids'].shape)
            if steps % 100 == 0:  # for some fun inspection
                print('REWARD_INPUT:', reward_tokenizer.batch_decode(t['input_ids'])[0])
                print("QUERY:", batch['query'][-1], "RESPONSE:", batch['response'][-1], r.logits[0])

            batch['rewards'].append(r.logits[0])
        # Compute sentiment score NOTE that we are putting the query + response in the same format as training the reward model
        # Run PPO step

        stats = ppo_trainer.step([_.squeeze() for _ in batch["input_ids"]], response_tensors, batch['rewards'])
        ppo_trainer.log_stats(stats, batch, batch['rewards'])

        steps += 1
        if steps % 250 == 0:
            print(batch)
            model.save_pretrained(f'./sawyer_rl/step_{steps}')



  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3639 [00:00<?, ?it/s]

REWARD_INPUT: <s>Who is Tina Turner

Tina Turner (born Anna Mae Bullock; November 26, 1939) is an American-born and naturalized Swiss retired singer, dancer and actress. Widely referred to as the "Queen of Rock 'n' Roll", she rose to prominence as the lead singer of the Ike & Tina Turner Revue before launching a successful career as a solo performer.

Turner began her career with Ike Turner's Kings of Rhythm in 1957. Under the name Little Ann, she appeared on her first record, "Boxtop", in 1958. In 1960, she debuted as Tina Turner with the hit duet single "A Fool in Love". The duo Ike & Tina Turner became "one of the most formidable live acts in history". They released hits such as "It's Gonna Work Out Fine", "River Deep – Mountain High", "Proud Mary", and "Nutbush City Limits" before disbanding in 1976.</s></s> Tina Turner played a significant role in establishing and strengthening the international pop scene, playing a significant role in promoting the empowerment and empowerment of 

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


REWARD_INPUT: <s>Which is a species of fish? Swordtail or Excalibur</s></s> Swordtails and Excalibur are not native to the Pacific Northwest, so it is not a species of fish.</s>
QUERY:  Which is a species of fish? Swordtail or Excalibur
 RESPONSE:  Swordtails and Excalibur are not native to the Pacific Northwest, so it is not a species of fish. tensor([-0.0232], grad_fn=<SelectBackward0>)
REWARD_INPUT: <s>What field of science (biology, physics or chemistry) was Marie Curie a researcher in?</s></s> She is a member of the French Physics Department of the University of Lausanne and is a former member of the Institut des Sciences de France (ISA). Additionally, she is a member of the Université de Guelph in Québec City for the period between 1919-1926.</s>
QUERY:  What field of science (biology, physics or chemistry) was Marie Curie a researcher in?
 RESPONSE:  She is a member of the French Physics Department of the University of Lausanne and is a former member of the Institut des Sciences

...

REWARD_INPUT: <s>Name some of the top basketball players</s></s> The
 The Electric</s>
QUERY:  Name some of the top basketball players
 RESPONSE:  The
 The Electric tensor([-2.7822], grad_fn=<SelectBackward0>)
{'response': [' The', " The'Tracker", ' The solitary', ' The freshly'], 'input_ids': [tensor([[50257,  1148,  3463, 30510,   922,   329,   345,    30,   198, 50258]],
       device='cuda:0'), tensor([[50257,  1867,   561,  2728, 33769, 26925,   284,  2728,   287, 45888,
           287,  9135,  4560,    30,   198,   198,   464, 33769, 26925,   318,
           257,  2099,   286, 26925,  9030,   326,   468,   587,  5495,   355,
           257,  2829,  3721,   284,  4620, 27111,  9135,  5339, 24048,  1871,
         24580,  7767,    13,  4285,   485, 26925, 12031,   284,  4726,  3746,
         31935,   257,  8271,   329,   262,  9478,   286,  3210,   640,    12,
            82,   677,   274,   357, 40972,   388,     8,   287,   257,  6977,
            11,   326, 17706, 27458,   664,  

REWARD_INPUT: <s>What is the best thing about being a manager?</s></s> The scars</s>
QUERY:  What is the best thing about being a manager?
 RESPONSE:  The scars tensor([-3.0761], grad_fn=<SelectBackward0>)
REWARD_INPUT: <s>Name the moon rocket that NASA is going to launch thru Space Launch System vehicle</s></s> The project</s>
QUERY:  Name the moon rocket that NASA is going to launch thru Space Launch System vehicle
 RESPONSE:  The project tensor([-2.4740], grad_fn=<SelectBackward0>)
REWARD_INPUT: <s>What is "Cien años de soledad"?

One Hundred Years of Solitude is a novel by Colombian writer Gabriel García Márquez, winner of the Nobel Prize for Literature in 1982. It is considered a masterpiece of Spanish-American and universal literature, as well as one of the most translated and read works in Spanish. 1 It was listed as one of the most important works of the Spanish language during the IV International Congress of the Spanish Language held in Cartagena de Indias in March 2007. 2 It

wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.


REWARD_INPUT: <s>What are some Australian craft beer brands that I should try?</s></s> The endless Rak</s>
QUERY:  What are some Australian craft beer brands that I should try?
 RESPONSE:  The endless Rak tensor([-1.0854], grad_fn=<SelectBackward0>)
REWARD_INPUT: <s>What is the best NFL football team of all time?</s></s> To flashbacks</s>
QUERY:  What is the best NFL football team of all time?
 RESPONSE:  To flashbacks tensor([-0.9270], grad_fn=<SelectBackward0>)
REWARD_INPUT: <s>What are some great progressive rock bands?</s></s> The</s>
QUERY:  What are some great progressive rock bands?
 RESPONSE:  The tensor([-3.1222], grad_fn=<SelectBackward0>)
REWARD_INPUT: <s>Can I feed table food to dogs?</s></s> In Mel</s>
QUERY:  Can I feed table food to dogs?
 RESPONSE:  In Mel tensor([-0.5325], grad_fn=<SelectBackward0>)
{'response': [' The Chip', ' That Children', ' The Oaks', ' Theslow'], 'input_ids': [tensor([[50257,  7343,   477,   262,  1524, 12815,   287,   262,  1708,  7322,
        

REWARD_INPUT: <s>What international trophy did the Club Africain football team won in 1971?

Club Africain (Arabic: النادي الإفريقي) is a Tunisian football club based in Tunis that competes in the Tunisian Championship. It fields several sports teams, including football, handball, basketball, swimming and volleyball. The football team was the first Tunisian club to win an international trophy, when they won the Maghreb Cup Winners Cup in 1971. Twenty years later, in 1991, Club Africain became the first Tunisian team to win the African Champions League.</s></s> The adoption</s>
QUERY:  What international trophy did the Club Africain football team won in 1971?

Club Africain (Arabic: النادي الإفريقي) is a Tunisian football club based in Tunis that competes in the Tunisian Championship. It fields several sports teams, including football, handball, basketball, swimming and volleyball. The football team was the first Tunisian club to win an international trophy, when they won the Maghreb Cu

In [ ]:
model.save_pretrained('./sawyer_rl')